# Дерево решений с использованием логистической регрессии в качестве разделяющей плоскости

> Для корректной работы ссылок оглавления лучше смотреть проект здесь \
> https://nbviewer.org/github/experiment0/experiments/blob/main/decision_tree_with_logistic_regression/index.ipynb

## Оглавление
- [Дерево решений для регрессии](#regression_tree)
    - [Загрузка данных](#rt_load)
    - [Построение дерева решений с помощью собственного класса `DecisionTreeRegressor`](#rt_custom)
    - [Построение дерева решений с помощью класса из `sklearn.tree.DecisionTreeRegressor`](#rt_sklearn)
- [Дерево решений для классификации](#classification_tree)
    - [Загрузка данных](#ct_load)
    - [Построение дерева решений с помощью собственного класса `DecisionTreeClassifier`](#ct_custom)
    - [Построение дерева решений с помощью класса из `sklearn.tree.DecisionTreeClassifier`](#ct_sklearn)
- [Дерево решений для бинарной классификации с использованием логистической регрессии](#logistic_tree)
- [Сравнение значений взвешенной неоднородности в первых вершинах полученных моделей для классификации](#weighted_impurity)
- [Вывод](#result)

In [1]:
from sklearn import tree
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.datasets import fetch_openml

from classes.DecisionTree import (
    DecisionTreeRegressor, 
    DecisionTreeClassifier,
)
from classes.DecisionTreeWithLogisticRegression import (
    DecisionTreeWithLogisticRegression
)

import warnings
warnings.filterwarnings("ignore")

> **Примечание**. При выполнении поиска оптимальных параметров сплита \
возможно возникновение такой ситуации, когда взвешенная неоднородность для двух разных наборов параметров \
будет одинаковой. \
В собственной реализации такие ситуации не учитываются: \
выбирается первый встретившийся вариант параметров с наименьшей взвешенной неоднородностью.
>
> В `sklearn` такие случаи обрабатываются следующим образом: \
из всех наборов параметров разбиения, для которых неоднородность после сплита минимальна из возможных \
и при этом одинакова, случайным образом выбирается только один этих наборов. \
Поэтому иногда деревья, полученные с помощью кода из собственного класса и деревья из `sklearn` могут не совпадать. \
Для получения этого совпадения при работе с деревьями из sklearn задается параметр `random_state`.

## Дерево решений для регрессии <a id="regression_tree"></a>

### Загрузка данных <a id="rt_load"></a>

In [2]:
# Для примера возьмем данные о производительности компьютеров
cpu_data_full = fetch_openml(name='machine_cpu')
cpu_data = cpu_data_full['frame']
cpu_data.head()

,MYCT,MMIN,MMAX,CACH,CHMIN,CHMAX,class
0,125.0,256.0,6000.0,256.0,16.0,128.0,198.0
1,29.0,8000.0,32000.0,32.0,8.0,32.0,269.0
2,29.0,8000.0,32000.0,32.0,8.0,32.0,220.0
3,29.0,8000.0,32000.0,32.0,8.0,32.0,172.0
4,29.0,8000.0,16000.0,32.0,8.0,16.0,132.0


**Описание признаков**
- `MYCT` - время цикла машины в наносекундах (целое число)
- `MMIN` - минимальный объем основной памяти в килобайтах (целое число)
- `MMAX` - максимальный объем основной памяти в килобайтах (целое число)
- `CACH` - кэш-память в килобайтах (целое число)
- `CHMIN` - минимальный объем каналов в единицах (целое число)
- `CHMAX` - максимальный объем каналов в единицах (целое число)
- `class` - опубликованная относительная производительность (целое число) (целевая переменная)

In [3]:
# Посмотрим на характеристики признаков
cpu_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 209 entries, 0 to 208
Data columns (total 7 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   MYCT    209 non-null    float64
 1   MMIN    209 non-null    float64
 2   MMAX    209 non-null    float64
 3   CACH    209 non-null    float64
 4   CHMIN   209 non-null    float64
 5   CHMAX   209 non-null    float64
 6   class   209 non-null    float64
dtypes: float64(7)
memory usage: 11.6 KB


Все данные числовые, пропусков нет.

In [4]:
# признаки
X = cpu_data.drop(['class'], axis = 1)
# целевой признак
y = cpu_data['class']

# разделяем на тренировочную и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

### Построение дерева решений с помощью собственного класса `DecisionTreeRegressor` <a id="rt_custom"></a>

In [5]:
# создаем объект класса для построения модели дерева решений
dtr_custom_model = DecisionTreeRegressor(max_depth=3)

# обучаем
dtr_custom_model.fit(X_train, y_train)
# печатаем дерево
dtr_custom_model.print_decision_tree()

# делаем предсказание
y_pred = dtr_custom_model.predict(X_test)
# выводим метрику MAPE (Mean Absolute Percent Error)
print()
print('MAPE score:', metrics.mean_absolute_percentage_error(y_test, y_pred))

|--- feature_2 <= 22485.00
|   |--- feature_3 <= 27.00
|   |   |--- feature_2 <= 10000.00
|   |   |   |--- value: [32.32142857142857]
|   |   |--- feature_2 > 10000.00
|   |   |   |--- value: [71.31818181818181]
|   |--- feature_3 > 27.00
|   |   |--- feature_3 <= 96.50
|   |   |   |--- value: [106.41666666666667]
|   |   |--- feature_3 > 96.50
|   |   |   |--- value: [231.4]
|--- feature_2 > 22485.00
|   |--- feature_4 <= 14.00
|   |   |--- feature_4 <= 7.00
|   |   |   |--- value: [143.6]
|   |   |--- feature_4 > 7.00
|   |   |   |--- value: [252.7]
|   |--- feature_4 > 14.00
|   |   |--- feature_2 <= 48000.00
|   |   |   |--- value: [433.6]
|   |   |--- feature_2 > 48000.00
|   |   |   |--- value: [636.0]

MAPE score: 0.5664946432356037


### Построение дерева решений с помощью класса из `sklearn.tree.DecisionTreeRegressor` <a id="rt_sklearn"></a>

In [6]:
# инициализируем модель дерева решений
dtr_model = tree.DecisionTreeRegressor(
    max_depth=3,
    criterion='squared_error', # критерий информативности
    random_state=0 # генератор случайных чисел для совпадения результатов
)

# обучаем
dtr_model.fit(X_train, y_train)

# выводим дерево решений на экран в виде списка условий
print(tree.export_text(decision_tree=dtr_model))

# Вычисляем значения информативности признаков
#print(dtr_model.feature_importances_)

# делаем предсказание
y_pred = dtr_model.predict(X_test)
# выводим метрику MAPE (Mean Absolute Percent Error)
print()
print('MAPE score:', metrics.mean_absolute_percentage_error(y_test, y_pred))

|--- feature_2 <= 22485.00
|   |--- feature_3 <= 27.00
|   |   |--- feature_2 <= 10000.00
|   |   |   |--- value: [32.32]
|   |   |--- feature_2 >  10000.00
|   |   |   |--- value: [71.32]
|   |--- feature_3 >  27.00
|   |   |--- feature_3 <= 96.50
|   |   |   |--- value: [106.42]
|   |   |--- feature_3 >  96.50
|   |   |   |--- value: [231.40]
|--- feature_2 >  22485.00
|   |--- feature_4 <= 14.00
|   |   |--- feature_4 <= 7.00
|   |   |   |--- value: [143.60]
|   |   |--- feature_4 >  7.00
|   |   |   |--- value: [252.70]
|   |--- feature_4 >  14.00
|   |   |--- feature_2 <= 48000.00
|   |   |   |--- value: [433.60]
|   |   |--- feature_2 >  48000.00
|   |   |   |--- value: [636.00]


MAPE score: 0.5664946432356037


> Получились одинаковые деревья решений и одинаковая метрика `MAPE`. \
Из чего можно сделать вывод, что собственный алгоритм для регрессии реализован в первом приближении корректно.

## Дерево решений для классификации <a id="classification_tree"></a>

### Загрузка данных <a id="ct_load"></a>

In [7]:
# Для примера возьмем данные об анализах крови на редкое генетическое заболевание
biomed_data_full = fetch_openml(name='biomed')
biomed_data = biomed_data_full['frame']
biomed_data.head()

,Observation_number,Hospital_identification_number_for_blood_sample,Age_of_patient,Date_that_blood_sample_was_taken,ml,m2,m3,m4,class
0,1,1027,30,100078,167.0,89.0,25.6,364.0,carrier
1,1,1013,41,100078,104.0,81.0,26.8,245.0,carrier
2,1,1324,22,80079,30.0,108.0,8.8,284.0,carrier
3,2,1332,22,80079,44.0,104.0,17.4,172.0,carrier
4,1,966,20,100078,65.0,87.0,23.8,198.0,carrier


**Описание признаков**

- `Observation_number` - номер наблюдения
- `Hospital_identification_number_for_blood_sample` - идентификационный номер больницы для образца крови
- `Age_of_patient` - возраст пациента
- `Date_that_blood_sample_was_taken` - дата взятия образца крови
- `M1` - сывороточная креатинкиназа.
- `M2` - гемопексин.
- `M3` - пируваткиназа.
- `M4` - лактатдегидрогеназа.

In [8]:
# для удалим лишние признаки
biomed_data.drop(
    columns=[
        'Observation_number', 
        'Hospital_identification_number_for_blood_sample',
        'Date_that_blood_sample_was_taken',
    ], 
    inplace=True
)

# удалим дубликаты
biomed_data.dropna(inplace=True)

# переведем целевой признак в числовой
biomed_data['class'] = biomed_data['class'].apply(lambda value: 1 if value == 'carrier' else 0)
biomed_data['class'] = biomed_data['class'].astype('int')

In [9]:
# Посмотрим на характеристики признаков
biomed_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 194 entries, 0 to 208
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Age_of_patient  194 non-null    int64  
 1   ml              194 non-null    float64
 2   m2              194 non-null    float64
 3   m3              194 non-null    float64
 4   m4              194 non-null    float64
 5   class           194 non-null    int32  
dtypes: float64(4), int32(1), int64(1)
memory usage: 9.9 KB


Все признаки числовые, пропусков нет.

In [10]:
# признаки
X = biomed_data.drop(['class'], axis = 1)
# целевой признак
y = biomed_data['class']

# разделяем на тренировочную и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

### Построение дерева решений с помощью собственного класса `DecisionTreeClassifier` <a id="ct_custom"></a>

In [11]:
# создаем объект класса для построения модели дерева решений
dtc_custom_model = DecisionTreeClassifier(max_depth=3)

# обучаем
dtc_custom_model.fit(X_train, y_train)
# печатаем дерево
dtc_custom_model.print_decision_tree()

# делаем предсказание
y_pred = dtc_custom_model.predict(X_test)
# выводим общий отчет по метрикам
print()
print(metrics.classification_report(y_test, y_pred))

|--- feature_1 <= 56.00
|   |--- feature_0 <= 40.50
|   |   |--- feature_4 <= 269.00
|   |   |   |--- value: [0]
|   |   |--- feature_4 > 269.00
|   |   |   |--- value: [1]
|   |--- feature_0 > 40.50
|   |   |--- value: [1]
|--- feature_1 > 56.00
|   |--- feature_2 <= 83.15
|   |   |--- feature_4 <= 219.00
|   |   |   |--- value: [0]
|   |   |--- feature_4 > 219.00
|   |   |   |--- value: [1]
|   |--- feature_2 > 83.15
|   |   |--- value: [1]

              precision    recall  f1-score   support

           0       0.89      0.89      0.89        35
           1       0.71      0.71      0.71        14

    accuracy                           0.84        49
   macro avg       0.80      0.80      0.80        49
weighted avg       0.84      0.84      0.84        49



### Построение дерева решений с помощью класса из `sklearn.tree.DecisionTreeClassifier` <a id="ct_sklearn"></a>

In [12]:
# инициализируем модель дерева решений
dtc_model = tree.DecisionTreeClassifier(
    max_depth=3,
    criterion='entropy', # критерий информативности
    random_state=42 # генератор случайных чисел для совпадения результатов
)

# обучаем
dtc_model.fit(X_train, y_train)

# выводим дерево решений на экран в виде списка условий
print(tree.export_text(decision_tree=dtc_model))

# делаем предсказание
y_pred = dtc_model.predict(X_test)
# выводим общий отчет по метрикам
print()
print(metrics.classification_report(y_test, y_pred))

|--- feature_1 <= 56.00
|   |--- feature_0 <= 40.50
|   |   |--- feature_4 <= 269.00
|   |   |   |--- class: 0
|   |   |--- feature_4 >  269.00
|   |   |   |--- class: 1
|   |--- feature_0 >  40.50
|   |   |--- class: 1
|--- feature_1 >  56.00
|   |--- feature_2 <= 83.15
|   |   |--- feature_4 <= 219.00
|   |   |   |--- class: 0
|   |   |--- feature_4 >  219.00
|   |   |   |--- class: 1
|   |--- feature_2 >  83.15
|   |   |--- class: 1


              precision    recall  f1-score   support

           0       0.89      0.89      0.89        35
           1       0.71      0.71      0.71        14

    accuracy                           0.84        49
   macro avg       0.80      0.80      0.80        49
weighted avg       0.84      0.84      0.84        49



> Получились одинаковые деревья решений и одинаковые метрики. \
Из чего можно сделать вывод, что собственный алгоритм для классификации реализован в первом приближении корректно.

## Дерево решений для бинарной классификации с использованием логистической регрессии <a id="logistic_tree"></a>

В классе `./classes/DecisionTreeWithLogisticRegression.py` \
реализован алгоритм дерева решения для бинарной классификации, \
аналогично, как в классе `./classes/DecisionTree.py`

Но в качестве плоскости, разделяющей выборки на 2 части,\
взят не предикат, а логистическая регрессия.

Посмотрим, какой это даст результат.

In [13]:
# создаем объект класса модели
dtlr_model = DecisionTreeWithLogisticRegression(max_depth=3)

# обучаем
dtlr_model.fit(X_train, y_train)
# печатаем дерево
dtlr_model.print_decision_tree()

# делаем предсказание
y_pred = dtlr_model.predict(X_test)
# выводим общий отчет по метрикам
print()
print(metrics.classification_report(y_test, y_pred))

|                  |                  |                  |--> value: 0; samples: 94; impurity: 0.342
|                  |                  |    samples: 94
|                  |                  |--> impurity: 0.342
|                  |                  |    predict: 0
|                  |                  |                  |--> None
|                  |    samples: 97
|                  |--> impurity: 0.446
|                  |    predict: 0
|                  |                  |--> value: 1; samples: 3; impurity: -0.000
|    samples: 145
|--> impurity: 0.947
|    predict: 0
|                  |                  |--> value: 0; samples: 3; impurity: -0.000
|                  |    samples: 48
|                  |--> impurity: 0.414
|                  |    predict: 1
|                  |                  |                  |--> value: 0; samples: 1; impurity: -0.000
|                  |                  |    samples: 45
|                  |                  |--> impurity: 0.154
|       

При той же глубине дерева мы получили лучшие метрики, чем для классического дерева решений.

## Сравнение значений взвешенной неоднородности в первых вершинах полученных моделей для классификации <a id="weighted_impurity"></a>

Сравним значения **взвешенной неоднородности** $G(Q,\ p)$ в результате деления первой вершины каждого из полученных деревьев.

Взвешенную неоднородность считаем по следующей формуле.

$$ G(Q,\ p)=\frac{N^{left}}{N}H(Q^{left})+\frac{N^{right}}{N}H(Q^{right}) $$

Где 
- $p$ - это условие деления выборки \
(предикат для классического дерева решений \
и обученная линейная регрессия для нашего эксперимента).
- $Q = {(x, y)}$ - это множество из объектов  выборки (строк $x$) и ответом к ним $y$.
- $N = |Q|$ - мощность этого множества (количество элементов  в нем).
- По условию $p$ выборка разбивается на 2 части: $Q^{left}$ и $Q^{right}$. \
Полученные выборки соответственно имеют мощности (количество элементов) $N^{left}$ и $N^{right}$.
- $H$ - это функция, по которой рассчитывается критерий информативности.\
В нашем случае для дерева классификации это энтропия Шеннона.

Подсчет этого значения реализован в методе `__calculate_weighted_impurity` каждого из классов\
(`DecisionTreeClassifier` и `DecisionTreeWithLogisticRegression`) и подсчитывается для каждой вершины.

Выведем значение только для первой вершины каждого дерева.\
Потому что гипотеза состоит в том, что с помощью линейной регрессии разделение будет лучше, чем с помощью предиката.\
И проверить мы это можем по первой вершине (далее разделение идет на другие выборки поэтому сравнение по внутренним вершинам будет некорректно).

In [14]:
print('Взвешенная неоднородность после деления первой вершины дерева класса DecisionTreeClassifier')
print(dtc_custom_model.decision_tree.weighted_impurity)

Взвешенная неоднородность после деления первой вершины дерева класса DecisionTreeClassifier
0.6262600470433869


In [15]:
print('Взвешенная неоднородность после деления первой вершины дерева класса DecisionTreeWithLogisticRegression')
print(dtlr_model.decision_tree.weighted_impurity)

Взвешенная неоднородность после деления первой вершины дерева класса DecisionTreeWithLogisticRegression
0.43514101416319195


Видим, что для класса `DecisionTreeWithLogisticRegression` взвешенная неоднородность меньше, следовательно деление произведено лучше.

## Вывод <a id="result"></a>

Использование логистической регрессии в качестве условия для  разделения выборок при делении дерева дало лучший результат, чем условие деления по предикату.

Также мы получили лучшие метрики для эксперементального класса.

На большом объеме данных подобный подход будет скорее всего не оптимальным.\
Потому что для каждого узла нужно будет строить и хранить обученную логистическую регрессию и набор предсказаний.